In [4]:
import pandas as pd
from scipy.stats import gamma
import numpy as np
import matplotlib.pyplot as plt

## Network Initialization

In [5]:
adjacency = pd.DataFrame(
    np.array([[1, 1, 1, 1, 0],
              [1, 1, 1, 1, 1],
              [1, 1, 1, 1, 1],
              [1, 1, 1, 1, 1],
              [0, 1, 1, 1, 1]]),
    columns=['1', '2', '3', '4', '5'],
    index=['1', '2', '3', '4', '5']
)
adjacency

,1,2,3,4,5
1,1,1,1,1,0
2,1,1,1,1,1
3,1,1,1,1,1
4,1,1,1,1,1
5,0,1,1,1,1


### Discretize the probability functions of all 9 edges

In [ ]:
# Create gamma distribution parameters
# Create range [0, T] with T = 4 to discretize our range of time when generating gamma distributions
x = np.linspace(0, 4, 4001)
# Create alpha and scale parameters for each edge
gamma_params = pd.DataFrame(
    columns=['1', '2', '3', '4', '5'],
    index=['1', '2', '3', '4', '5']
)

gamma_params['1']['2'] = gamma_params['2']['1'] = (1, 1)
gamma_params['1']['3'] = gamma_params['3']['1'] = (1, 1)

# Create gamma distributions for each edge
gamma_distributions = pd.DataFrame(
    columns=['1', '2', '3', '4', '5'],
    index=['1', '2', '3', '4', '5']
)

for i in gamma_distributions.index:
    for j in gamma_distributions.columns:
        alpha, beta = gamma_params[j][i]
        gamma_distributions[j][i] = gamma.pdf(x, alpha, scale=1/beta)

,1,2,3,4,5
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN


## Simulation